# ノードの停止

---

CoursewareHub環境のノードを一時的に停止します。

> このNotebookは AWS, AzureのVCノードに対してのみ利用可能です。他のクラウドプロバイダで実行中のVCノードはノード停止の機能をVCPがサポートしていないためエラーとなります。

## パラメータの指定

ノードの停止を行うのに必要となるパラメータを入力します。

### UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

UnitGroupの変数をgroup_varsファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

指定されたUnitGroupでworkerノードの状態を変更するスケジュール設定が行われている場合は、このnotebookでノードの停止を行うことができません。「956-workerノード操作のスケジュールを削除する.ipynb」でスケジュール設定を削除した後にこのnotebookを実行してください。

In [ ]:
if "schedule" in gvars:
    raise RuntimeError("スケジュール設定が行われています")

### VCCアクセストークンの入力

VCノードを停止するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## ノードの停止

現在のノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

### workerノードの停止

全てのworkerノードを停止します。

In [ ]:
unit_worker = ug.get_unit('worker')
unit_worker.power_off_nodes(num_power_off_nodes=gvars['worker_nodes'])

workerノードの状態を確認します。全ノードの `node_state` の表示が `POWER_OFF`となっていることを確認してください。

In [ ]:
unit_worker.df_nodes()

一部のVCノードが `POWER_ON`のままの場合は、次のセルのコメントを外して実行してください。その際に対象とするVCノードの`node_no`の値を指定してください。`node_no`の値は上のセルの出力結果で確認してください。

In [ ]:
# node_no =
# unit_worker.power_off_nodes(node_no=node_no)
# unit_worker.df_nodes()

### managerノードの停止

managerノードを停止します。

> workerノードのみを停止したい場合はこの節を実行しないでください。

In [ ]:
unit_manager = ug.get_unit('manager')
unit_manager.power_off_nodes()

managerノードの状態を確認します。`node_state` の表示が `POWER_OFF`となっていることを確認してください。

In [ ]:
unit_manager.df_nodes()